In [ ]:
from stock_article import StockArticle
from utility import HTMLParser
from utility import FileSaver

stock_article = StockArticle()

news = stock_article.fetch_news("Google", news_count=10)

html_parser = HTMLParser()
file_saver = FileSaver()
for item in news:
    html_parser.parse_html(item["link"])
    file_name = html_parser.get_title() + ".txt"
    parsed_content = html_parser.get_paragraphs()
    file_saver.save_to_file("google", parsed_content, file_name)


In [32]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.quiverquant.com/congresstrading/politician/Nancy%20Pelosi-P000197"
response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

trade_data = []
for script in soup.find_all("script"):
    if script.string and "let tradeData" in script.string:
        trade_data_script = script.string
        break


trade_data_json = re.search(r"let tradeData = (\[.*?\]);", trade_data_script, re.DOTALL).group(1)
trade_data_json = trade_data_json.replace('null', 'None').replace('NaN', 'float("nan")')
trade_data = eval(trade_data_json)

# Create a DataFrame
columns = ["Stock", "Transaction", "Filled", "Traded", "Description", "Change", "Politician", "ID", "Company", "Type", "Amount", "Chamber", "Party", "Sector", "Value"]
df = pd.DataFrame(trade_data, columns=columns)

df.to_excel("nancy_pelosi.xlsx", index=False)
